In [ ]:
# Couchbase expirements

In [1]:
# Install dependencies
!pip install couchbase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 20.1 MB/s eta 0:00:00


In [2]:
# Verify couchbase working
import couchbase
print(couchbase.__version__)

4.3.5


In [5]:
import socket

def get_colab_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        # Doesn't even have to be reachable
        s.connect(('10.255.255.255', 1))
        IP = s.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        s.close()
    return IP

colab_ip = get_colab_ip()
print(f"Colab's IP Address: {colab_ip}")

Colab's IP Address: 172.28.0.12


In [9]:
from google.colab import userdata
from couchbase.cluster import Cluster
from couchbase.auth import PasswordAuthenticator
from couchbase.options import ClusterOptions

# For test configuring Couchbase cluster online to accept incoming connection from anywhere (Warning: Never do this for actual setup!)
# Retrieve credentials from Google Secrets
try:
    username = userdata.get('COUCHBASE_USERNAME')
    password = userdata.get('COUCHBASE_PASSWORD')
    cluster_address = userdata.get('COUCHBASE_CLUSTER')
    bucket_name = userdata.get('COUCHBASE_BUCKET')
    scope_name = "inventory"
    collection_name = "airline"

    # Check if any of the secrets are None
    if not all([username, password, cluster_address, bucket_name]):
        raise ValueError("One or more Couchbase credentials not found in Google Secrets.")

except KeyError as e:
    raise KeyError(f"Required secret not found in Google Secrets: {e}")

# Connect to Couchbase
try:
    cluster = Cluster(
        cluster_address,
        ClusterOptions(PasswordAuthenticator(username, password))
    )

    bucket = cluster.bucket(bucket_name)
    print("Successfully connected to Couchbase!")

    # Example: Access the default collection
    default_collection = bucket.default_collection()
    print("Successfully accessed the default collection.")

    # You can now perform Couchbase operations here
    # For example, to insert a document:
    # result = default_collection.insert("mydocument", {"name": "example"})
    # print(result)

except Exception as e:
    print(f"Error connecting to Couchbase: {e}")

Successfully connected to Couchbase!
Successfully accessed the default collection.


In [10]:
# Get a reference to a scope
scope = bucket.scope("inventory")

# Get a reference to a collection
collection = scope.collection("airline")

# N1QL query to limit the number of rows returned
query = f"SELECT META().id, * FROM `{bucket_name}`.`{scope_name}`.`{collection_name}` LIMIT 10"

try:
    # Execute the query
    result = cluster.query(query)

    # Iterate through the results and print the document ID and content
    for row in result:
        print(f"Document ID: {row['id']}")  # Access the document ID
        print(f"Document Content: {row}")  # Print the entire document content
        print("-" * 20)

except Exception as e:
    print(f"Error executing query: {e}")


Document ID: airline_10
Document Content: {'airline': {'callsign': 'MILE-AIR', 'country': 'United States', 'iata': 'Q5', 'icao': 'MLA', 'id': 10, 'name': '40-Mile Air', 'type': 'airline'}, 'id': 'airline_10'}
--------------------
Document ID: airline_10123
Document Content: {'airline': {'callsign': 'TXW', 'country': 'United States', 'iata': 'TQ', 'icao': 'TXW', 'id': 10123, 'name': 'Texas Wings', 'type': 'airline'}, 'id': 'airline_10123'}
--------------------
Document ID: airline_10226
Document Content: {'airline': {'callsign': 'atifly', 'country': 'United States', 'iata': 'A1', 'icao': 'A1F', 'id': 10226, 'name': 'Atifly', 'type': 'airline'}, 'id': 'airline_10226'}
--------------------
Document ID: airline_10642
Document Content: {'airline': {'callsign': None, 'country': 'United Kingdom', 'iata': None, 'icao': 'JRB', 'id': 10642, 'name': 'Jc royal.britannica', 'type': 'airline'}, 'id': 'airline_10642'}
--------------------
Document ID: airline_10748
Document Content: {'airline': {'cal